In [3]:
def convert_to_square(image, new_size=None):
    if len(image.shape)!=2:
        image = image[:, :, 0]
    square_size = np.max(image.shape)
    h, w = image.shape[0], image.shape[1]
    delta_w, delta_h = square_size - w, square_size - h
    left, top = delta_w//2, delta_h//2
    square_image = np.median(image)*np.ones((h+delta_h, w+delta_w))
    
    
    square_image[top:top+h, left:left+w] = image.copy()
    
    # Blurring borders
    kernel_size = 5
    kernel = np.ones((kernel_size,kernel_size),np.float32)/(kernel_size*kernel_size)
    square_image_blured = cv2.filter2D(square_image,-1,kernel)
    mask = np.zeros_like(square_image)
    padding = 2
    val = 1
    if top==0:
        mask[0:h, left-padding:left+padding] = val
        mask[0:h, left+w-padding:left+w+padding] = val
    else:
        mask[top-padding:top+padding, 0:w] = val
        mask[top+h-padding:top+h+padding, 0:w] = val
    square_image[mask==1]=square_image_blured[mask==1]
    
    if not new_size is None:
        square_image = cv2.resize(square_image, dsize=(new_size, new_size), interpolation=cv2.INTER_CUBIC)
    return square_image